# Power your products with ChatGPT and your own data

This is a walkthrough taking readers through how to build starter Q&A and Chatbot applications using the ChatGPT API and their own data. 

It is laid out in these sections:
- **Setup:** 
    - Initiate variables and source the data
- **Lay the foundations:**
    - Set up the vector database to accept vectors and data
    - Load the dataset, chunk the data up for embedding and store in the vector database
- **Make it a product:**
    - Add a retrieval step where users provide queries and we return the most relevant entries
    - Summarise search results with GPT-3
    - Test out this basic Q&A app in Streamlit
- **Build your moat:**
    - Create an Assistant class to manage context and interact with our bot
    - Use the Chatbot to answer questions using semantic search context
    - Test out this basic Chatbot app in Streamlit
    
Upon completion, you have the building blocks to create your own production chatbot or Q&A application using OpenAI APIs and a vector database.

This notebook was originally presented with [these slides](https://drive.google.com/file/d/1dB-RQhZC_Q1iAsHkNNdkqtxxXqYODFYy/view?usp=share_link), which provide visual context for this journey.

In [1]:
# load OpenAI API Key from `.env` file
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key  = os.getenv('OPENAI_API_KEY')

## Setup

First we'll setup our libraries and environment variables

In [2]:
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract
from numpy import array, average

from database import get_redis_connection

# Set our default models and chunking size
from config import COMPLETIONS_MODEL, EMBEDDINGS_MODEL, CHAT_MODEL, TEXT_EMBEDDING_CHUNK_SIZE, VECTOR_FIELD_NAME

# Ignore unclosed SSL socket warnings - optional in case you get these errors
#import warnings

#warnings.filterwarnings(action="ignore", message="unclosed", category=ImportWarning)
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

#pd.set_option('display.max_colwidth', 0)

In [3]:
data_dir = os.path.join(os.curdir,'data')
pdf_files = sorted([x for x in os.listdir(data_dir) if 'DS_Store' not in x])
pdf_files

['Laws of the Game 2023_24.pdf',
 'Official-Playing-Rules-2022-23-NBA-Season.pdf']

## Laying the foundations

### Storage

We're going to use Redis as our database for both document contents and the vector embeddings. You will need the full Redis Stack to enable use of Redisearch, which is the module that allows semantic search - more detail is in the [docs for Redis Stack](https://redis.io/docs/stack/get-started/install/docker/).

To set this up locally, you will need to install Docker and then run the following command: ```docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest```.

The code used here draws heavily on [this repo](https://github.com/RedisAI/vecsim-demo).

After setting up the Docker instance of Redis Stack, you can follow the below instructions to initiate a Redis connection and create a Hierarchical Navigable Small World (HNSW) index for semantic search.

In [4]:
# Setup Redis
from redis import Redis
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField,
    NumericField
)
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)

redis_client = get_redis_connection()

In [5]:
# Constants
VECTOR_DIM = 1536 #len(data['title_vector'][0]) # length of the vectors
#VECTOR_NUMBER = len(data)                 # initial number of vectors
PREFIX = "sportsdoc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [6]:
# Create search index

# Index
INDEX_NAME = "sports-index"           # name of the search index
VECTOR_FIELD_NAME = 'content_vector'

# Define RediSearch fields for each of the columns in the dataset
# This is where you should add any additional metadata you want to capture
filename = TextField("filename")
text_chunk = TextField("text_chunk")
file_chunk_index = NumericField("file_chunk_index")

# define RediSearch vector fields to use HNSW index

text_embedding = VectorField(VECTOR_FIELD_NAME,
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC
    }
)
# Add all our field objects to a list to be created as an index

fields = [filename,text_chunk,file_chunk_index,text_embedding]

In [7]:
redis_client.ping()

True

In [8]:
# Optional step to drop the index if it already exists
#redis_client.ft(INDEX_NAME).dropindex()

# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except Exception as e:
    print(e)
    # Create RediSearch Index
    print('Not there yet. Creating')
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

Unknown Index name
Not there yet. Creating


### Ingestion

We'll load up our PDFs and do the following
- Initiate our tokenizer
- Run a processing pipeline to:
    - Mine the text from each PDF
    - Split them into chunks and embed them
    - Store them in Redis

In [9]:
# The transformers.py file contains all of the transforming functions, including ones to chunk, embed and load data
# For more details, check the file and work through each function individually
from transformers import handle_file_string

In [10]:
# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Process each PDF file and prepare for embedding
for pdf_file in pdf_files:
    
    pdf_path = os.path.join(data_dir, pdf_file)
    print(pdf_path)
    
    # Extract the raw text from each PDF using textract
    text = textract.process(pdf_path, method='pdfminer')
    
    # Chunk each document, embed the contents and load to Redis
    handle_file_string((pdf_file, text.decode("utf-8")), tokenizer, redis_client, VECTOR_FIELD_NAME, INDEX_NAME)

./data/Laws of the Game 2023_24.pdf
./data/Official-Playing-Rules-2022-23-NBA-Season.pdf


In [11]:
# Check that our docs have been inserted
redis_client.ft(INDEX_NAME).info()['num_docs']

'271'

## Make it a product

Now we can test that our search works as intended by:
- Querying our data in Redis using semantic search and verifying results
- Adding a step to pass the results to GPT-3 for summarisation

In [12]:
from database import get_redis_results
pd.set_option('display.max_colwidth', 0)  # this enables showing all result in the cell

In [13]:
user_query='what is a penalty kick'

result_df = get_redis_results(redis_client, user_query, index_name=INDEX_NAME)
result_df.head()  # top k is set to 2 by default in database.py

,id,result,certainty
0,0,"• the player taking the penalty kick or a team-mate offends: •if the ball enters the goal, the kick is retaken • if the ball does not enter the goal, the referee stops play and restarts with an indirect free kick except for the following when play will be stopped and restarted with an indirect free kick, regardless of whether or not a goal is scored: •a penalty kick is kicked backwards • a team-mate of the identified kicker takes the kick the referee cautions the player who took the kick • feinting to kick the ball once the kicker has completed the run-up (feinting in the run-up is permitted) the referee cautions the kicker • the goalkeeper offends: •if the ball enters the goal, a goal is awarded If, before the ball is in play, one of the following occurs: 120 • if the ball misses the goal or rebounds from the crossbar or goalpost(s), the kick is only retaken if the goalkeeper’s offence clearly impacted on the kicker • if the ball is prevented from entering the goal by the goalkeeper, the kick is retaken If the goalkeeper’s offence results in the kick being retaken, the goalkeeper is warned for the first offence in the game and cautioned for any subsequent offence(s) in the game • a team-mate of the goalkeeper offends: •if the ball enters the goal, a goal is awarded •if the ball does not enter the goal, the kick is retaken • a player of both teams offends, the kick is retaken unless a player commits a more serious offence (e.g.",0.162130534649
1,1,"The players other than the kicker and goalkeeper must be: • at least 9.15 m (10 yds) from the penalty mark • behind the penalty mark • inside the field of play • outside the penalty area After the players have taken positions in accordance with this Law, the referee signals for the penalty kick to be taken. The player taking the penalty kick must kick the ball forward backheeling is permitted provided the ball moves forward. When the ball is kicked, the defending goalkeeper must have at least part of one foot touching, in line with, or behind, the goal line. The ball is in play when it is kicked and clearly moves. 119 14 Laws of the Game 2023/24 | Law 14 | The Penalty Kick 2. Offences and sanctions The kicker must not play the ball again until it has touched another player. The penalty kick is completed when the ball stops moving, goes out of play or the referee stops play for any offence. Additional time is allowed for a penalty kick to be taken and completed at the end of each half of the match or extra time. When additional time is allowed, the penalty kick is completed when, after the kick has been taken, the ball stops moving, goes out of play, is played by any player (including the kicker) other than the defending goalkeeper, or the referee stops play for an offence by the kicker or the kicker’s team. If a defending team player (including the goalkeeper) commits an offence and the penalty is missed/saved, the penalty is retaken. Once the referee has signalled for a penalty kick to be taken, the kick must be taken if it is not taken, the referee may take disciplinary action before signalling again for the kick to be taken.",0.1637185812


In [14]:
# Build a prompt to provide the original query, the result and ask to summarise for the user
summary_prompt = '''Summarise this result in a bulleted list to answer the search query a customer has sent.
Search query: SEARCH_QUERY_HERE
Search result: SEARCH_RESULT_HERE
Summary:
'''
summary_prepped = summary_prompt.replace('SEARCH_QUERY_HERE', user_query).replace('SEARCH_RESULT_HERE', result_df['result'][0])  # only choose the top 1
summary = openai.Completion.create(engine=COMPLETIONS_MODEL, prompt=summary_prepped, max_tokens=500)

# Response provided by GPT-3
print(summary['choices'][0]['text'])

• Penalty kick is a kick given to a team when a foul is committed within the penalty area.
• It can be taken by the player that was fouled or a team-mate.
• If the ball enters the goal, the kick is retaken if the offense affected the kicker, otherwise a goal is awarded.
• If the ball does not enter the goal, an indirect free kick is awarded, regardless of whether or not a goal is scored.
• If the goalie or a team-mate offends, the kick is retaken unless a more serious offense is committed (e.g. aggression).


### Search

Now that we've got our knowledge embedded and stored in Redis, we can now create an internal search application. Its not sophisticated but it'll get the job done for us.

In the directory containing this app, execute ```streamlit run app-search.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data.

__Example Questions__:
- what is a penalty kick?
- What does it mean when a player is ejected in basketball?

## Build your moat

The Q&A was useful, but fairly limited in the complexity of interaction we can have - if the user asks a sub-optimal question, there is no assistance from the system to prompt them for more info or conversation to lead them down the right path.

For the next step we'll make a Chatbot using the Chat Completions endpoint, which will:
- Be given instructions on how it should act and what the goals of its users are
- Be supplied some required information that it needs to collect
- Go back and forth with the customer until it has populated that information
- Say a trigger word that will kick off semantic search and summarisation of the response

For more details on our Chat Completions endpoint and how to interact with it, please check out the docs [here](https://platform.openai.com/docs/guides/chat).

### Framework

This section outlines a basic framework for working with the API and storing context of previous conversation "turns". Once this is established, we'll extend it to use our retrieval endpoint.

In [15]:
# A basic example of how to interact with our ChatCompletion endpoint
# It requires a list of "messages", consisting of a "role" (one of system, user or assistant) and "content"
question = 'How can you help me'


completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": question}
  ]
)
print(f"{completion['choices'][0]['message']['role']}: {completion['choices'][0]['message']['content']}")

assistant: I can help you in several ways:

1. Answering questions: I can provide information and answer any queries you may have to the best of my knowledge.

2. Providing suggestions: If you're looking for ideas or recommendations, I can offer suggestions based on your needs or preferences.

3. Giving instructions: If you need directions or guidelines for a particular task, I can provide step-by-step instructions to assist you.

4. Offering support: If you're going through a difficult time or need someone to talk to, I can provide emotional support and lend a listening ear.

5. Assisting with research: If you need help finding specific information or conducting research on a particular topic, I can help gather relevant data and resources.

6. Language support: I can aid in translation, grammar, or language-related questions if you need assistance with communication.

These are just a few ways I can help, but feel free to ask anything else, and I'll do my best to assist you.


In [16]:
from termcolor import colored

# A basic class to create a message as a dict for chat
class Message:
    
    
    def __init__(self,role,content):
        
        self.role = role
        self.content = content
        
    def message(self):
        
        return {"role": self.role,"content": self.content}
        
# Our assistant class we'll use to converse with the bot
class Assistant:
    
    def __init__(self):
        self.conversation_history = []

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=prompt
            )
            
            response_message = Message(completion['choices'][0]['message']['role'], completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'

    def ask_assistant(self, next_user_prompt, colorize_assistant_replies=True):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        self.conversation_history.append(assistant_response)
        return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                print(output)

In [17]:
# Initiate our Assistant class
conversation = Assistant()

# Create a list to hold our messages and insert both a system message to guide behaviour and our first user question
messages = []
system_message = Message('system','You are a helpful business assistant who has innovative ideas')
user_message = Message('user','What can you do to help me')
messages.append(system_message.message())
messages.append(user_message.message())
messages

[{'role': 'system',
  'content': 'You are a helpful business assistant who has innovative ideas'},
 {'role': 'user', 'content': 'What can you do to help me'}]

In [18]:
# Get back a response from the Chatbot to our question
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

As a helpful business assistant, I can provide a wide range of assistance to help you grow and succeed in your business. Here are a few ways I can help:

1. Idea generation: I can brainstorm innovative ideas for product development, marketing strategies, customer engagement, and more. These ideas can help you differentiate your business from competitors and capture new opportunities.

2. Market research: I can conduct thorough market research to gather data and insights about your target audience, industry trends, competitor analysis, and emerging markets. This information can help you make informed decisions and stay ahead of the competition.

3. Business planning: I can assist you in creating a comprehensive business plan that outlines your objectives, strategies, financial projections, and action steps. This plan will serve as a roadmap for your business and guide your decision-making process.

4. Process optimization: I can analyze your existing business processes and identify area

In [19]:
next_question = 'Tell me more about option 2'

# Initiate a fresh messages list and insert our next question
messages = []
user_message = Message('user', next_question)
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

Certainly! Market research is a crucial component of any business strategy as it helps you gain insights into your target market, understand customer needs and preferences, analyze industry trends, and identify opportunities for growth. Here's a deeper dive into how I can help with market research:

1. Target audience analysis: I can help you identify and define your target audience by conducting thorough demographic, psychographic, and behavioral research. This includes analyzing factors such as age, gender, location, interests, purchasing behavior, and more. Understanding your target audience enables you to tailor your products, services, and marketing efforts to better meet their needs.

2. Competitive analysis: I can conduct in-depth research on your competitors, analyzing factors such as their products or services, pricing, marketing strategies, strengths, weaknesses, and market positioning. This competitive intelligence helps you understand your industry landscape, identify gaps 

In [20]:
next_question = 'which option I asked you about?'

# Initiate a fresh messages list and insert our next question
messages = []
user_message = Message('user', next_question)
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

Apologies for the confusion. Option 2 refers to conducting thorough market research. Here's a summary of how I can assist you with market research:

I can help you gather data and insights about your target audience, industry trends, competitor analysis, and emerging markets. This information is crucial for making informed decisions and staying ahead of the competition. I assist in:

1. Target audience analysis: Defining and understanding your target audience in terms of demographics, psychographics, and behavior.

2. Competitive analysis: Researching your competitors, their offerings, pricing, marketing strategies, strengths, and weaknesses.

3. Industry trends and market analysis: Providing up-to-date information on industry trends, consumer preferences, and market dynamics.

4. Customer feedback and surveys: Designing and conducting customer surveys to gather insights on satisfaction, preferences, and pain points.

5. Market segmentation: Segmenting your target market based on demog

In [21]:
# Print out a log of our conversation so far
conversation.pretty_print_conversation_history()

user:
What can you do to help me
assistant:
As a helpful business assistant, I can provide a wide range of assistance to help you grow and succeed in your business. Here are a few ways I can help:

1. Idea generation: I can brainstorm innovative ideas for product development, marketing strategies, customer engagement, and more. These ideas can help you differentiate your business from competitors and capture new opportunities.

2. Market research: I can conduct thorough market research to gather data and insights about your target audience, industry trends, competitor analysis, and emerging markets. This information can help you make informed decisions and stay ahead of the competition.

3. Business planning: I can assist you in creating a comprehensive business plan that outlines your objectives, strategies, financial projections, and action steps. This plan will serve as a roadmap for your business and guide your decision-making process.

4. Process optimization: I can analyze your e

### Knowledge retrieval

Now we'll extend the class to call a downstream service when a stop sequence is spoken by the Chatbot.

The main changes are:
- The system message is more comprehensive, giving criteria for the Chatbot to advance the conversation
- Adding an explicit stop sequence for it to use when it has the info it needs
- Extending the class with a function ```_get_search_results``` which sources Redis results

In [22]:
# Updated system prompt requiring Question and sport to be extracted from the user
system_prompt = '''
You are a helpful sports knowledge base assistant. You need to capture a Question and Year from each customer.
The Question is their query on sports rule, and the Sport is the sport of concern, which is either basketball or soccer.
Think about this step by step:
- The user will ask a Question
- You will ask them for the Sport if their question didn't include a Sport
- Once you have the Sport, say "searching for answers".

Example:

User: I'd like to know how many players on each team

Assistant: Certainly, what sport would you like this for?

User: basketball please.

Assistant: Searching for answers.
'''

# New Assistant class to add a vector database call to its responses
class RetrievalAssistant:
    
    def __init__(self):
        self.conversation_history = []  

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model=CHAT_MODEL,
              messages=prompt,
              temperature=0.1
            )
            
            response_message = Message(completion['choices'][0]['message']['role'], completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'
    
    # The function to retrieve Redis search results
    def _get_search_results(self, prompt):
        latest_question = prompt
        search_content = get_redis_results(redis_client, latest_question, INDEX_NAME)['result'][0]
        return search_content
        

    def ask_assistant(self, next_user_prompt):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        
        # Answer normally unless the trigger sequence is used "searching_for_answers"
        if 'searching for answers' in assistant_response['content'].lower():
            print('search function call triggered')
            question_extract = openai.Completion.create(model=COMPLETIONS_MODEL, prompt=f"Extract the user's latest question and the sport for that question from this conversation: {self.conversation_history}. Extract it as a sentence stating the Question and Sport")
            search_result = self._get_search_results(question_extract['choices'][0]['text'])
            print('search result', search_result)
            
            # We insert an extra system prompt here to give fresh context to the Chatbot on how to use the Redis results
            # In this instance we add it to the conversation history, but in production it may be better to hide
            self.conversation_history.insert(-1, {"role": 'system', "content": f"Answer the user's question using this content: {search_result}. If you cannot answer the question, say 'Sorry, I don't know the answer to this one'"})
            
            #[self.conversation_history.append(x) for x in next_user_prompt]
            print(self.conversation_history)
            
            assistant_response = self._get_assistant_response(self.conversation_history)
            #print(next_user_prompt)
            #print(assistant_response)
            self.conversation_history.append(assistant_response)
            return assistant_response
        else:
            print('no search function is triggered')
            self.conversation_history.append(assistant_response)
            return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                #prefix = entry['role']
                print(output)

In [23]:
conversation = RetrievalAssistant()
messages = []
system_message = Message('system', system_prompt)
user_message = Message('user', 'how many players on each team')
messages.append(system_message.message())
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
response_message

no search function is triggered


{'role': 'assistant',
 'content': 'Certainly, what sport would you like this for?'}

In [24]:
messages = []
user_message = Message('user', 'basketball please.')
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
response_message

search function call triggered
search result    Additional balls which meet the requirements of Law 2 may be placed around   the field of play and their use is under the referee’s control.    3. Additional balls    42    The Players    1. Number of players     A match is played by two teams, each with a maximum of eleven players    one must be the goalkeeper. A match may not start or continue if either team   has fewer than seven players.   If a team has fewer than seven players because one or more players has   deliberately left the field of play, the referee is not obliged to stop play and   the advantage may be played, but the match must not resume after the ball has   gone out of play if a team does not have the minimum number of seven players.   If the competition rules state that all players and substitutes must be named   before kick-off and a team starts a match with fewer than eleven players,   only the players and substitutes named on the team list may take part in the   mat

{'role': 'assistant', 'content': 'Searching for answers.'}

In [25]:
conversation.pretty_print_conversation_history()

user:
how many players on each team
assistant:
Certainly, what sport would you like this for?
user:
basketball please.
assistant:
Searching for answers.


### Chatbot

Now we'll put all this into action with a real (basic) Chatbot.

In the directory containing this app, execute ```streamlit run app-chat.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data. 


### Consolidation

Over the course of this notebook you have:
- Laid the foundations of your product by embedding our knowledge base
- Created a Q&A application to serve basic use cases
- Extended this to be an interactive Chatbot

These are the foundational building blocks of any Q&A or Chat application using our APIs - these are your starting point, and we look forward to seeing what you build with them!